In [ ]:
!pip install transformers[torch]

In [ ]:
pip install sumy

In [ ]:

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer, AdamW
import os

import nltk
from torch.utils.data import DataLoader
import torch
from torch.nn import functional as F




# Tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:

from nltk.stem import WordNetLemmatizer


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Preprocessing function
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Removing punctuation
    text = re.sub(r'[^\w\s.]', '', text)
    #      # removing digits
    # text = re.sub(r'\d', '', text)

    # Removing stopwords
    stop_words = set(nltk.corpus.stopwords.words('english'))
    tokens = nltk.word_tokenize(text)
    text = ' '.join([token for token in tokens if token not in stop_words])
    # Remove extra spaces
    text = re.sub(' +', ' ', text)

    # Remove extra spaces
    text = re.sub(' +', ' ', text)
    return text

# # Specify the file paths for one specific pair of input and label
# file_path_annual = '/content/drive/MyDrive/ProjectNLP/Annual_reports/17.txt'
# file_path_summary = '/content/drive/MyDrive/ProjectNLP/gold_summaries/17_1.txt'


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# trained_model.to(device)
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

In [ ]:
def summarize_report(report_path):
    report_content = open(report_path, 'r', encoding='utf-8').read().strip()
    report_content = preprocess_text(report_content)

    sentences = nltk.tokenize.sent_tokenize(report_content)
    length = 0
    chunk = ""
    chunks = []
    count = -1

    for sentence in sentences:
        count += 1
        combined_length = len(tokenizer.tokenize(sentence)) + length

        if combined_length <= 1024:
            chunk += sentence + " "
            length = combined_length

            if count == len(sentences) - 1:
                chunks.append(chunk.strip())
        else:
            chunks.append(chunk.strip())
            length = 0
            chunk = ""
            chunk += sentence + " "
            length = len(tokenizer.tokenize(sentence))

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # trained_model.to(device)
    inputs = [tokenizer(chunk, return_tensors="pt", max_length=1024, truncation=True, padding='max_length') for chunk in chunks]

    chunk_summaries = []
    for input in inputs:
        # Ensure input tensor is on the same device as the model
        input = {key: value.to(device) for key, value in input.items()}
        output = model.generate(**input)

        generated_summary = tokenizer.decode(output[0], skip_special_tokens=True)
        chunk_summaries.append(generated_summary)

    final_summary = ' '.join(chunk_summaries)
    return final_summary

# Path to the folder containing annual reports
reports_folder = "/content/drive/MyDrive/Rest_files"
# Path to the target folder to save generated summaries
target_folder = "/content/drive/MyDrive/Bart_summary"
index = 600

# Iterate through all files in the reports folder
for filename in os.listdir(reports_folder):
    if filename.endswith(".txt"):
        index += 1
        report_path = os.path.join(reports_folder, filename)
        generated_summary = summarize_report(report_path)

        # Save the generated summary to the target folder with the same name as the input report
        output_file_path = os.path.join(target_folder, f"{filename.split('.')[0]}_{index}.txt")
        with open(output_file_path, "w", encoding="utf-8") as output_file:
            output_file.write(generated_summary)

        print(f"Generated summary for {filename}")

print("All reports summarized and saved.")

Generated summary for 31076.txt
Generated summary for 31114.txt


In [ ]:
import os

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

def lexrank_summarize(text, num_sentences):
    summarizer = LexRankSummarizer()
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summary = summarizer(parser.document, num_sentences)
    summary_sentences = [sentence._text for sentence in summary]
    return ' '.join(summary_sentences)
    # Combine sentences into a paragraph
    summary_paragraph = ' '.join(summary_sentences)

    return summary_paragraph


def process_files(input_folder, output_folder, num_sentences=50):
    # Ensure the output folder exists, create if not
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process each .txt file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".txt"):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.txt")

            with open(input_path, 'r', encoding='utf-8') as file:
                input_text = file.read()
            preprocces_input=preprocess_text(input_text)
            # Apply LexRank summarization
            summary_result = lexrank_summarize(preprocces_input, num_sentences)

            # Save the summary to the output file
            with open(output_path, 'w', encoding='utf-8') as output_file:
                output_file.write(summary_result)
                print ("summary Ganerated {filename}")

if __name__ == "__main__":
    input_folder_path = "/content/drive/MyDrive/Bart_summary"
    output_folder_path = "/content/drive/MyDrive/Bart_summary/Finale_summary"
    num_sentences_for_summary =50

    process_files(input_folder_path, output_folder_path, num_sentences_for_summary)

summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ganerated
summary Ga

In [2]:
import pandas as pd
from io import StringIO

# Your CSV-like text data
csv_data = "/content/drive/MyDrive/Result_Bart_lexrank_main.csv"

# Create a pandas DataFrame from the CSV-like text
df = pd.read_csv(csv_data)

# Group by 'ROUGE-Type' and calculate the average for each ROUGE type
averages = df.groupby('ROUGE-Type').mean().reset_index()

# Print the resulting DataFrame
print(averages)

                           ROUGE-Type     Task Name  Avg_Recall  \
0    ROUGE-1+StopWordRemoval+Stemming  30910.759036    0.323981   
1    ROUGE-2+StopWordRemoval+Stemming  30910.759036    0.109059   
2    ROUGE-L+StopWordRemoval+Stemming  30910.759036    0.327257   
3  ROUGE-SU4+StopWordRemoval+Stemming  30910.759036    0.168824   

   Avg_Precision  Avg_F-Score  Num Reference Summaries  
0       0.203830     0.244204                 3.445783  
1       0.049646     0.065568                 3.445783  
2       0.256650     0.283103                 3.445783  
3       0.069800     0.095886                 3.445783  


<ipython-input-2-0f437ba2c2c5>:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  averages = df.groupby('ROUGE-Type').mean().reset_index()
